## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-27-22-49-48 +0000,nyt,At Least 3 Killed in Arizona Flood That Swept ...,https://www.nytimes.com/2025/09/27/us/arizona-...
1,2025-09-27-22-48-35 +0000,nypost,3 dead after severe Arizona floods grip small ...,https://nypost.com/2025/09/27/us-news/arizona-...
2,2025-09-27-22-40-12 +0000,nyt,Trader Joe’s and Walmart Frozen Meals May Have...,https://www.nytimes.com/2025/09/27/us/listeria...
3,2025-09-27-22-38-55 +0000,nyt,Trump Displays Little Interest in Fostering Un...,https://www.nytimes.com/2025/09/27/us/politics...
4,2025-09-27-22-24-42 +0000,nypost,Netanyahu expecting to make ‘painful and signi...,https://nypost.com/2025/09/27/world-news/netan...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2372/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
24,trump,52
381,up,10
543,president,9
71,new,9
55,china,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
152,2025-09-27-06-45-39 +0000,nypost,US to revoke Colombian president’s visa after ...,https://nypost.com/2025/09/27/us-news/us-to-re...,99
171,2025-09-27-02-03-00 +0000,wsj,The Supreme Court on Friday granted the Trump ...,https://www.wsj.com/politics/policy/supreme-co...,99
43,2025-09-27-18-56-46 +0000,nyt,"Trump Says He Has Ordered Troops to Portland, ...",https://www.nytimes.com/2025/09/27/us/politics...,89
77,2025-09-27-15-38-00 +0000,wsj,Trump Administration Asks Supreme Court to Uph...,https://www.wsj.com/us-news/law/trump-administ...,87
86,2025-09-27-14-19-43 +0000,cbc,Trump administration asks U.S. Supreme Court t...,https://www.cbc.ca/news/world/trump-supreme-co...,87


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
152,99,2025-09-27-06-45-39 +0000,nypost,US to revoke Colombian president’s visa after ...,https://nypost.com/2025/09/27/us-news/us-to-re...
173,50,2025-09-27-02-00-00 +0000,wsj,A U.S. plan to field thousands of cutting-edge...,https://www.wsj.com/politics/national-security...
171,47,2025-09-27-02-03-00 +0000,wsj,The Supreme Court on Friday granted the Trump ...,https://www.wsj.com/politics/policy/supreme-co...
15,34,2025-09-27-21-27-07 +0000,nypost,Anti-ICE protesters led by paid activists marc...,https://nypost.com/2025/09/27/us-news/brooklyn...
181,31,2025-09-27-01-00-00 +0000,wsj,The indictment of James Comey sealed the remar...,https://www.wsj.com/politics/policy/trump-over...
63,28,2025-09-27-17-01-42 +0000,nypost,Trump rages at bombshell report alleging FBI h...,https://nypost.com/2025/09/27/us-news/trump-ra...
51,28,2025-09-27-18-06-21 +0000,bbc,Israeli strikes kill dozens in Gaza including ...,https://www.bbc.com/news/articles/c87y58jgn5lo...
69,27,2025-09-27-16-35-58 +0000,latimes,D4Vd's abandoned Tesla was ticketed 11 days be...,https://www.latimes.com/california/story/2025-...
76,26,2025-09-27-15-46-19 +0000,nypost,Renaissance Faire ‘Lord Mayor’ sues ex-lover f...,https://nypost.com/2025/09/27/us-news/lady-lor...
106,25,2025-09-27-12-00-00 +0000,latimes,"Lopez: Nine months after the Eaton fire, 'mome...",https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
